### Functions

In [6]:
from __future__ import print_function
import os
import urllib
import json
from collections import defaultdict
import pandas as pd

# Pulls data from EIA API.
def get_data_from_EIA(APIKey, seriesID):
    url = url = 'http://api.eia.gov/series/?api_key=' + APIKey + '&series_id=' + seriesID
    response = urllib.request.urlopen(url)    
    data = json.loads(response.read().decode('utf8'))
    return data

# Prints JSON data to text file.
def write_json_to_txt(jsonObj, filename):
    with open(filename, "w") as outfile:
        json.dump(jsonObj, outfile, indent=4)

# Pull data for all states into a nested dictionary.
def compile_state_energy_data():
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
              "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    totalEnergyProdByState = dict(keys = states)
    for state in states:
        dataFromEIA = get_data_from_EIA("DC4B181A26EDF82B41F2A7446102CAB7","SEDS.TEPRB.%s.A" % state)
        totalEnergyProdByState[state] = dataFromEIA
    return totalEnergyProdByState

# Get the range of years with recorded data.
def get_years(totalEnergyProdByState):
    datasetSample = totalEnergyProdByState['AK']['series'][0]['data']
    years = [item[0] for item in datasetSample]
    return years

# Collect data and reorganize by year.
def state_energy_by_year(totalEnergyProdByState):
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
              "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    years = get_years(totalEnergyProdByState)
    
    stateDict = dict((states,0) for states in states)
    stateEnergyByYear = dict((year,0) for year in years)
    
    for year in years:
        stateEnergyByYear[year] = stateDict
        
    energyByStateSimple = stateDict
    for state in states:
        energyByStateSimple[state] = dict(totalEnergyProdByState[state]['series'][0]['data'])  
        
    flipped = defaultdict(dict)
    for key, val in energyByStateSimple.items():
        for subkey, subval in val.items():
            flipped[subkey][key] = subval
            
    stateEnergyByYear = dict(flipped)
    
    return stateEnergyByYear

### View Data

In [9]:
totalEnergyProdByState = compile_state_energy_data()
stateEnergyByYear = state_energy_by_year(totalEnergyProdByState)

In [11]:
stateEnergyByYear['1960']

{'AK': 21554,
 'AL': 474229,
 'AR': 289353,
 'AZ': 36737,
 'CA': 2630898,
 'CO': 481962,
 'CT': 17406,
 'DC': 155,
 'DE': 4982,
 'FL': 37861,
 'GA': 96116,
 'HI': 292,
 'IA': 37943,
 'ID': 77688,
 'IL': 1526429,
 'IN': 441137,
 'KS': 1361353,
 'KY': 1846570,
 'LA': 5071351,
 'MA': 53748,
 'MD': 61535,
 'ME': 59784,
 'MI': 174400,
 'MN': 34957,
 'MO': 102979,
 'MS': 526044,
 'MT': 289631,
 'NC': 127507,
 'ND': 197457,
 'NE': 169409,
 'NH': 25629,
 'NJ': 20514,
 'NM': 1562831,
 'NV': 22267,
 'NY': 204924,
 'OH': 901866,
 'OK': 2072617,
 'OR': 190489,
 'PA': 2391326,
 'RI': 2951,
 'SC': 81969,
 'SD': 15866,
 'TN': 287803,
 'TX': 12075806,
 'UT': 389446,
 'VA': 824357,
 'VT': 17339,
 'WA': 431849,
 'WI': 64968,
 'WV': 3237864,
 'WY': 1018174}